In [ ]:
!pip install keras_nlp keras_core

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras_core as keras
import keras_nlp
import seaborn as sns
import matplotlib.pyplot as plt


print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)
print("KerasNLP version:", keras_nlp.__version__)

In [ ]:
import kagglehub

# Download latest version
path_daigt_v4_train_dataset = kagglehub.dataset_download("thedrcat/daigt-v4-train-dataset")

print("Path to dataset files:", path_daigt_v4_train_dataset)

In [ ]:
daigt_v4_train_datase = pd.read_csv(path_daigt_v4_train_dataset+'/train_v4_drcat_01.csv')
daigt_v4_train_datase.head()

In [ ]:
import matplotlib.pyplot as plt

# Fonction pour afficher un graphique en camembert pour chaque dataset
def plot_pie_chart(ax, train_data, label, title):
    label_counts = train_data[label].value_counts()
    label_counts.plot.pie(autopct='%1.1f%%', startangle=90, cmap='Set3', ax=ax)
    ax.set_title(title)
    ax.set_ylabel('')  # Supprimer le label de l'axe y

# Créer un tableau de graphiques 2x2
fig, axs = plt.subplots(3, 2, figsize=(16, 12))

# Afficher les graphiques en camembert pour chaque dataset
plot_pie_chart(axs[0,0], daigt_v4_train_datase,'label', 'Distribution des valeurs dans la colonne '+ 'generated')
plot_pie_chart(axs[0,1], daigt_v4_train_datase,'RDizzl3_seven', 'Distribution des valeurs dans la colonne generated')
plot_pie_chart(axs[1,0], daigt_v4_train_datase,'model', 'Distribution des valeurs dans la colonne model')
plot_pie_chart(axs[1,1], daigt_v4_train_datase,'source', 'Distribution des valeurs dans la colonne source')
plot_pie_chart(axs[2,0], daigt_v4_train_datase,'prompt_name', 'Distribution des valeurs dans la colonne prompt_name')
# Ajuster les espaces entre les graphiques
plt.tight_layout()

# Afficher les graphiques
plt.show()

In [ ]:
# prompt: caculer les valeur unique dans colone source et model

print("Unique values in 'source' column:", daigt_v4_train_datase['source'].unique())
print("\nUnique values in 'model' column:", daigt_v4_train_datase['model'].unique())
print(" nbr Unique values in 'source' column:", len(daigt_v4_train_datase['source'].unique()))
print("\n nbr Unique values in 'model' column:", len(daigt_v4_train_datase['model'].unique()))

In [ ]:
daigt_v4_train_datase.rename(columns = {"label":"generated"}, inplace=True)

In [ ]:
daigt_v4_train_datase.head()

In [ ]:
train = pd.read_csv('/content/llm_dataset/train_essays.csv')
train.head()

In [ ]:
df_train_essays_final = pd.concat([train[["text", "generated"]], daigt_v4_train_datase[["text", "generated"]]])

df_train_essays_final.info()

In [ ]:
# We choose 512 because it's the limit of DistilBert
SEQ_LENGTH = 512

# Use a shorter sequence length.
preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(
    "distil_bert_base_en_uncased",
    sequence_length=SEQ_LENGTH,
)

# Pretrained classifier.
classifier = keras_nlp.models.DistilBertClassifier.from_preset(
    "distil_bert_base_en_uncased",
    num_classes=2,
    activation=None,
    preprocessor=preprocessor,
)

# Re-compile (e.g., with a new learning rate)
classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="adam",
    metrics=[
        keras.metrics.SparseCategoricalAccuracy()
   ]
)


# Access backbone programmatically (e.g., to change `trainable`).
classifier.backbone.trainable = False


classifier.summary()

In [ ]:
# Split the dataset into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train_essays_final["text"],
                                                    df_train_essays_final["generated"],
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
# Fit
classifier.fit(x=X_train,
               y=y_train,
               validation_data=(X_test, y_test),
               epochs=10,
               batch_size=64)